In [ ]:
import matlab.engine
import numpy as np;
from sklearn.neighbors import KDTree 
from FactorBP import *
from FactorBP.FactorGraph import *
from scipy.spatial import Delaunay
import scipy.io as sio
from Utils import *
eng = matlab.engine.start_matlab()

In [ ]:
def CreateTensorCar(P1,P2,bpermute):
    res = {}
    NP1 = P1.shape[0]
    NP2 = P2.shape[0]
    if(bpermute):
        res['GT'] = np.random.permutation(P2.shape[0])
    else:
        res['GT'] = np.array(range(P2.shape[0]))
    P1 = P1[res['GT'], :]
    
    nT = NP1 * NP2
    
    t1 = np.floor(np.random.rand(3, nT) * NP1)
    while(True):
        probFound=False;
        for i in range(3):
            ind=(t1[i,:]==t1[(i+1)%3,:])
        if(np.sum(ind)!=0):
            idxs = np.nonzero(ind)
            t1[i][idxs]=np.floor(np.random.rand(1,len(idxs))*NP1);
            probFound=True;
        if(probFound == False):
            break;
    
    tri1 = Delaunay(P1)
    tri2 = Delaunay(P2)
    t1 = tri1.simplices
    t2 = tri2.simplices
    Feature = computeFeature(P1, P2, t1)
    
    kdt = KDTree(Feature['feat2'], metric='euclidean')
    nNN = 2000;
    [dist, indices] = kdt.query(Feature['feat1'], k=nNN, return_distance=True)
    
    dist = np.exp(- (dist / np.mean(dist) ))
    
    res['Triplets'] = t1
    res['NTriplets'] = indices
    res['Similarity'] = dist
    return res

def ComputeAccuracy(X1, GT, NofInliers):
    Ccnt = 0
    for b in xrange(NofInliers):
        if(int(X1[b]) == int(GT[b])):
            Ccnt += 1
    res = Ccnt * 1.0 / GT.shape[0]
    return res
def IndicesToVec(indices, NofNodes, NofStates):
    res = VecInt(NofNodes)
    res[2] = indices % NofStates
    indices /= NofStates
    res[1] = indices % NofStates
    indices /= NofStates
    res[0] = indices
    return res

def computeFeature(P1, P2, T1 ):
    res = {}
    F1 = np.zeros([T1.shape[1], 3])
    NP2 = P2.shape[0]
    F2 = np.zeros([NP2 * NP2 * NP2, 3])
    for i in xrange(T1.shape[1]):
        F1[i] = computeFeatureSimple(P1, T1[:,i])
    Fcnt = 0
    for i in xrange(NP2):
        for j in xrange(NP2):
            for k in xrange(NP2):
                F2[Fcnt] = computeFeatureSimple(P2, [i,j,k])
                Fcnt = Fcnt + 1
    res['feat1'] = F1
    res['feat2'] = F2
    
    return res

def RunCar(CarData, idx, NofOus):
    car1 = CarData[idx]
    LocalFeature1 = car1['features1']
    LocalFeature2 = car1['features2']
    PT1 = LocalFeature1[:,0:2]
    PT2 = LocalFeature2[:,0:2]
    orientation1 = LocalFeature1[:,8]
    orientation2 = LocalFeature2[:,8]
    GT=car1['gTruth'][0]
    NofInliers = len(GT)
    
    NofNodes = NofInliers + NofOus
    KP = ComputeFeatureDistance(orientation1[0:NofNodes], orientation2[0:NofNodes])
    KTriplets = CreateTensorHouseDelaunay(PT1[0:NofNodes],PT2[0:NofNodes], False)
    indH1=np.array(range(NofNodes*NofNodes))
    valH1=KP.reshape((NofNodes * NofNodes, 1))
    KTriplets['indH1'] = indH1
    KTriplets['valH1'] = valH1
    
            
    
    sio.savemat('Temp.mat', KTriplets)
    
    KP = np.exp(-KP)
    NofStates = intArray(NofNodes)
    for i in xrange(NofNodes):
        NofStates[i] = NofNodes
    G = CFactorGraph(NofNodes, NofStates)
    bi = doubleArray(NofNodes)
    for ni in range(NofNodes):
        for xi in range(NofNodes):
            bi[xi] = float(KP[ni][xi]) 
    G.AddNodeBelief(ni, bi)
    for i in xrange(KTriplets['Triplets'].shape[0]):
        T = KTriplets['Triplets'][i]
        T1 = VecInt(3)
        T1[0] = int(T[0])
        T1[1] = int(T[1])
        T1[2] = int(T[2])
        NNZIs = KTriplets['NTriplets']
        NNZVs = KTriplets['Similarity'][i]
        NNZIVecs = VecVecInt(NNZIs.shape[0])
        NNZVVecs = doubleArray(NNZIs.shape[0])
        for xi in xrange(NNZIs.shape[0]):
            cNTriplets = VecInt(3)
            cNTriplets[0] = int(NNZIs[xi][0])
            cNTriplets[1] = int(NNZIs[xi][1])
            cNTriplets[2] = int(NNZIs[xi][2])
            #cNTriplets = IndicesToVec(NNZIs[xi], 3, NofNodes)
            NNZIVecs[xi] = cNTriplets
            NNZVVecs[xi] = NNZVs[xi]
        G.AddGenericGenericSparseFactor(T1, NNZIVecs, NNZVVecs)
    G.SetVerbose(True)
    #G.Solve(1000)
    G.AddAuctionFactor()
    G.Solve(10000)
    G.SetVerbose(False)
    res1 = BaBSolver(G, 50, 1000, 0.005, False)
    resBag = eng.runBcagm(nargout=3)
    resBagMP = eng.runBcagmQuad1(1,nargout=3)
    resBagIPFP = eng.runBcagmQuad1(2,nargout=3)
    resHGM = eng.runHGM(nargout=3)
    resRRWHM = eng.runRRWHM(nargout=3)
    resTM = eng.runTensorMatching(nargout=3)
    
    res = {};
    res[0] = res1
    res[1] = resBag
    res[2] = resBagMP
    res[3] = resBagIPFP
    res[4] = resHGM
    res[5] = resRRWHM
    res[6] = resTM
    res['GT'] = KTriplets['GT']
    res['NofInliers'] = NofInliers
    return res

In [ ]:

CarData = LoadCar()

NofAlgorithms = 7
AlgorithmNames=['Ours', 'BCA', 'BCA-MP', 'BCA-IPFP', 'HGM', 'RRWHM', 'TM']

NofPairs = 5
NofOusMax = 3

MeanAcc = np.zeros([NofOusMax,NofAlgorithms])
MeanRtime = np.zeros([NofOusMax, NofAlgorithms])
MeanObj = np.zeros([NofOusMax, NofAlgorithms])

for NofOus in xrange(1):
    
    Accuracy = np.zeros([NofAlgorithms, NofPairs])
    Rtime = np.zeros([NofAlgorithms, NofPairs])
    Obj = np.zeros([NofAlgorithms, NofPairs])
    
    for ImageI in xrange(2, 1 + NofPairs):
        res = RunCar(CarData, ImageI, NofOus)
        Accuracy[0][ImageI-1] = ComputeAccuracy(res[0].Decode, res['GT'], res['NofInliers'])
        Rtime[0][ImageI-1] = res[0].Time
        Obj[0][ImageI-1] = res[0].Value
        
        for AlgoI in xrange(1, NofAlgorithms):
            Accuracy[AlgoI][ImageI-1] = ComputeAccuracy(res[AlgoI][1][0], res['GT'], res['NofInliers'])
            Rtime[AlgoI][ImageI-1] = res[AlgoI][0]
            Obj[AlgoI][ImageI-1] = res[AlgoI][2]/6
        
        #MaxObj = np.max(Obj[:,ImageI-1])
        #Obj[:,ImageI-1] /= MaxObj

        for ai in xrange(NofAlgorithms):
            print('%s Accuracy %f Running Time %f Obj %f' %(AlgorithmNames[ai], 
                                                             Accuracy[ai][ImageI-1],
                                                            Rtime[ai][ImageI-1],
                                                            Obj[ai][ImageI-1]))
        print('Finished Frame: %d' %ImageI)
    
    for ai in xrange(NofAlgorithms):
        MeanAcc[NofOus][ai] = np.mean(Accuracy[ai])
        MeanRtime[NofOus][ai] = np.mean(Rtime[ai])
        MeanObj[NofOus][ai] = np.mean(Obj[ai])
        print('%s Accuracy %f Running Time %f Obj %f' %(AlgorithmNames[ai], 
                                                        np.mean(Accuracy[ai]),
                                                        np.mean(Rtime[ai]),
                                                        np.mean(Obj[ai])))
                         



In [ ]:
res = RunCar(CarData, 1, 10)


In [ ]:
res

In [ ]:
np.reshape.__doc__


In [ ]:

range(100)

In [ ]:
G.SetVerbose(True)
G.AddAuctionFactor()
G.Solve(1000)

In [ ]:
print(G.GetDecode())

In [ ]:
print(KTriplets['GT'])

In [ ]:
KTriplets['NTriplets'].shape

In [ ]:
KTriplets['Triplets'].shape

In [ ]:
print(PT1[0:NofInliers,:])


In [ ]:
print(PT2[0:NofInliers,:])

t2 = Delaunay(PT2[0:NofInliers,:])
t2.simplices

In [ ]:
28 * 6